In [ ]:
%%sql
MERGE INTO fact_sales AS target
USING (
    SELECT 
        fm.src_pk,
        fm.seq_per_store,
        -- Store & Date Information
        CAST(fm.store_id_from AS INT) AS store_id_from,
        CAST(fm.store_id_to AS INT) AS store_id_to,
        CAST(fm.product_id AS INT) AS product_id,
        fm.movement_date,

        -- Sales & Profit Metrics
        fm.sale_amt AS net_sales_w_taxes, 
        fm.sale_amt / (p.iva_tax_rate * p.ieps_tax_rate) AS net_sales_wo_tax, -- Net Sales Excluding Tax
        1 AS total_transactions, -- TODO: Use Distinct SEQ_ID from the store
        
        CASE 
            WHEN fm.movement_id = 'VT' THEN fm.sale_amt 
            ELSE 0 
        END AS gross_sales_w_taxes,

        CASE 
            WHEN fm.movement_id = 'VT' THEN fm.sale_amt / (p.iva_tax_rate * p.ieps_tax_rate) 
            ELSE 0 
        END AS gross_sales_wo_tax, -- Gross Sales Excluding Tax

        CASE 
            WHEN fm.movement_id = 'VT' THEN fm.qty 
            ELSE 0 
        END AS units_sold,

        fm.sale_amt + fm.cost_amt AS gross_profit_w_taxes,
        (fm.sale_amt / (p.iva_tax_rate * p.ieps_tax_rate)) + fm.cost_amt AS gross_profit_wo_tax, -- Gross Profit Excluding Tax
        
        CASE 
            WHEN fm.movement_id = 'VT' THEN fm.cost_amt 
            ELSE 0 
        END  AS cost_of_goods_sold

    FROM fmovarti_intermediate_silver fm
    LEFT JOIN dimproducts p 
        ON fm.product_id = p.src_product_id
    WHERE (fm.movement_id IN ('VT', 'DC') 
       OR (fm.movement_id = 'CA' AND fm.ReferenceDoc LIKE 'DC%'))
    AND CAST(fm.fabric_load_date AS DATE) >= DATEADD(DAY, -2, CURRENT_DATE)
) AS source
ON target.src_pk = source.src_pk
WHEN MATCHED THEN
    UPDATE SET 
        target.seq_per_store = source.seq_per_store,
        target.store_id_from = source.store_id_from,
        target.store_id_to = source.store_id_to,
        target.product_id = source.product_id,
        target.movement_date = source.movement_date,
        target.net_sales_w_taxes = source.net_sales_w_taxes,
        target.net_sales_wo_tax = source.net_sales_wo_tax,
        target.total_transactions = source.total_transactions,
        target.gross_sales_w_taxes = source.gross_sales_w_taxes,
        target.gross_sales_wo_tax = source.gross_sales_wo_tax,
        target.units_sold = source.units_sold,
        target.gross_profit_w_taxes = source.gross_profit_w_taxes,
        target.gross_profit_wo_tax = source.gross_profit_wo_tax,
        target.cost_of_goods_sold = source.cost_of_goods_sold
WHEN NOT MATCHED THEN
    INSERT (
        src_pk, seq_per_store, store_id_from, store_id_to, product_id, movement_date, 
        net_sales_w_taxes, net_sales_wo_tax, total_transactions, gross_sales_w_taxes, 
        gross_sales_wo_tax, units_sold, gross_profit_w_taxes, gross_profit_wo_tax, 
        cost_of_goods_sold
    )
    VALUES (
        source.src_pk, source.seq_per_store, source.store_id_from, source.store_id_to, source.product_id, source.movement_date, 
        source.net_sales_w_taxes, source.net_sales_wo_tax, source.total_transactions, source.gross_sales_w_taxes, 
        source.gross_sales_wo_tax, source.units_sold, source.gross_profit_w_taxes, source.gross_profit_wo_tax, 
        source.cost_of_goods_sold
    );
